<a href="https://colab.research.google.com/github/otanet/WebScraping_2023_sub/blob/main/GNews_1_%E9%80%94%E4%B8%AD_20230109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import requests
from bs4 import BeautifulSoup

res = requests.get('https://www.google.com/search?q=%E3%82%B9%E3%83%91%E3%82%A4%E3%83%80%E3%83%BC%E3%83%97%E3%83%A9%E3%82%B9&sxsrf=AJOqlzVdMcD-skixlwJ2n26gvo3dMnUGVg:1673090517752&source=lnms&tbm=nws&sa=X&ved=2ahUKEwigifPvq7X8AhXXMd4KHVfnAO4Q_AUoAXoECAEQAw&biw=1151&bih=510&dpr=1.65')
soup = BeautifulSoup(res.text)

article_list = soup.select('div > a jsname')

#for a in article_list:
#    print(a['href'], a.get_text())
print(article_list)

[]


### 20230109 1933-
### ニュース記事をスクレイピング - Qiita
### https://qiita.com/babmuclr/items/64a8833bfe7a7e32f09e


### 1. urllib.requestとBeautifulSoupを使う方法 2. Seleniumを使う方法


In [2]:
from urllib import request
from bs4 import BeautifulSoup

In [6]:
url = "https://www.google.com/search?q=%E3%82%B9%E3%83%91%E3%82%A4%E3%83%80%E3%83%BC%E3%83%97%E3%83%A9%E3%82%B9&sxsrf=AJOqlzXkTKJFytjRWtYTOJ9FP2W7Ec30Ew:1673260597332&source=lnms&sa=X&ved=2ahUKEwj23pS8pbr8AhXXZd4KHeccDBwQ_AUoAHoECAEQAg&biw=1151&bih=510&dpr=1.65"
url

'https://www.google.com/search?q=%E3%82%B9%E3%83%91%E3%82%A4%E3%83%80%E3%83%BC%E3%83%97%E3%83%A9%E3%82%B9&sxsrf=AJOqlzXkTKJFytjRWtYTOJ9FP2W7Ec30Ew:1673260597332&source=lnms&sa=X&ved=2ahUKEwj23pS8pbr8AhXXZd4KHeccDBwQ_AUoAHoECAEQAg&biw=1151&bih=510&dpr=1.65'

In [7]:
response = request.urlopen(url)


HTTPError: ignored

In [ ]:
soup = BeautifulSoup(response,"xml")
response.close()
articles = soup.find_all("item")

## 結論: bs4とrequestで、ダイレクトなURLからだと403エラーがでるので、Seleniumで、検索キーワードを入力するところからやるしかない。

### Pythonでグーグルニュースの検索結果をスクレイピング (2) Beautiful Soupを使う - Qiita
### https://qiita.com/tyamaguchi636/items/9b758b194057ac2df4f9
### 20230109 1958-

In [21]:
import pandas as pd    #スクレイピング結果をデータフレーム形式でcvsファイルに保存するため
import pprint    #データフレームの一部を表示するため
from bs4 import BeautifulSoup  #取得したWebページの情報の解析と抽出
import requests     #Webページの情報の取得
import urllib       #キーワードのurlエンコード取得

In [23]:
s = "スパイダープラス"
s_quote = urllib.parse.quote(s)
url_b4 = 'https://news.google.com/search?q=' + s_quote + '&hl=ja&gl=JP&ceid=JP%3Aja'

In [24]:
#検索結果ページの情報を取得
res = requests.get(url_b4)
soup = BeautifulSoup(res.content, "html.parser")

In [25]:
#すべての記事部の情報を選択
articles = soup.select(".iRPxbe")

In [28]:
print(articles)

[]


In [18]:
#各記事の情報をfor　～　enumerate分で繰り返し取得してリストに代入
news = list()   #代入のための空のリストを作成

for i, entry in enumerate(articles, 1):
    title = entry.find("h3").text
    summary = entry.find("span").text
    summary = title + "。" + summary
    #url_elm = entry.find("a")を下記に変更
    url_elm = entry.find("article")
    link = url_elm.get("jslog")
    link = link.lstrip("85008; 2:")     #左端削除
    link = link.rstrip("; track:click") #右端削除
    time_elm = entry.find("time")
    try:    #例外処理
        ymd = time_elm.get("datetime")
    except AttributeError:
        ymd = "0000-00-00"
    ymd = ymd[0:10]
    ymd = ymd.replace("-", "/")     #置換
    sortkey = ymd[0:4] + ymd[5:7] + ymd[8:10] #年月日でのソート用

    tmp = {             #辞書型で格納
        "title": title,
        "summary": summary,
        "link": link,
        "published": ymd,
        "sortkey": sortkey
        }

    news.append(tmp)  #各記事の情報をリストに追加

    #データフレームに変換してcsvファイルで保存
    news_df = pd.DataFrame(news)
    pprint.pprint(news_df.head())  #最初の5行を表示してデータ確認 
    filename = s + ".csv"
    news_df.to_csv(filename, encoding='utf-8-sig', index=False) 

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 必要なライブラリの呼び出し
import pandas as pd    #スクレイピング結果をデータフレーム形式でcvsファイルに保存するため
import pprint    #データフレームの一部を表示するため
from bs4 import BeautifulSoup  #取得したWebページの情報の解析と抽出
import requests     #Webページの情報の取得
import urllib       #キーワードのurlエンコード取得

#検索ワード「タピる」を文字変換して検索結果ページのurlの間に挿入
s = "タピる"
s_quote = urllib.parse.quote(s)
url_b4 = 'https://news.google.com/search?q=' + s_quote + '&hl=ja&gl=JP&ceid=JP%3Aja'

#検索結果ページの情報を取得
res = requests.get(url_b4)
soup = BeautifulSoup(res.content, "html.parser")

#すべての記事部の情報を選択
articles = soup.select(".xrnccd")

#各記事の情報をfor　～　enumerate分で繰り返し取得してリストに代入
news = list()   #代入のための空のリストを作成

for i, entry in enumerate(articles, 1):
    title = entry.find("h3").text
    summary = entry.find("span").text
    summary = title + "。" + summary
    #url_elm = entry.find("a")を下記に変更
    url_elm = entry.find("article")
    link = url_elm.get("jslog")
    link = link.lstrip("85008; 2:")     #左端削除
    link = link.rstrip("; track:click") #右端削除
    time_elm = entry.find("time")
    try:    #例外処理
        ymd = time_elm.get("datetime")
    except AttributeError:
        ymd = "0000-00-00"
    ymd = ymd[0:10]
    ymd = ymd.replace("-", "/")     #置換
    sortkey = ymd[0:4] + ymd[5:7] + ymd[8:10] #年月日でのソート用

    tmp = {             #辞書型で格納
        "title": title,
        "summary": summary,
        "link": link,
        "published": ymd,
        "sortkey": sortkey
        }

    news.append(tmp)  #各記事の情報をリストに追加

    #データフレームに変換してcsvファイルで保存
    news_df = pd.DataFrame(news)
    pprint.pprint(news_df.head())  #最初の5行を表示してデータ確認 
    filename = s + ".csv"
    news_df.to_csv(filename, encoding='utf-8-sig', index=False) 

In [ ]:
# 必要なライブラリの呼び出し
import pandas as pd    #スクレイピング結果をデータフレーム形式でcvsファイルに保存するため
import pprint    #データフレームの一部を表示するため
from bs4 import BeautifulSoup  #取得したWebページの情報の解析と抽出
import requests     #Webページの情報の取得
import urllib       #キーワードのurlエンコード取得

#検索ワード「タピる」を文字変換して検索結果ページのurlの間に挿入
s = "スパイダープラス"
s_quote = urllib.parse.quote(s)
url_b4 = 'https://news.google.com/search?q=' + s_quote + '&hl=ja&gl=JP&ceid=JP%3Aja'

#検索結果ページの情報を取得
res = requests.get(url_b4)
soup = BeautifulSoup(res.content, "html.parser")

#すべての記事部の情報を選択
articles = soup.select(".xrnccd")

#各記事の情報をfor　～　enumerate分で繰り返し取得してリストに代入
news = list()   #代入のための空のリストを作成

for i, entry in enumerate(articles, 1):
    title = entry.find("h3").text
    summary = entry.find("span").text
    summary = title + "。" + summary
    #url_elm = entry.find("a")を下記に変更
    url_elm = entry.find("article")
    link = url_elm.get("jslog")
    link = link.lstrip("85008; 2:")     #左端削除
    link = link.rstrip("; track:click") #右端削除
    time_elm = entry.find("time")
    try:    #例外処理
        ymd = time_elm.get("datetime")
    except AttributeError:
        ymd = "0000-00-00"
    ymd = ymd[0:10]
    ymd = ymd.replace("-", "/")     #置換
    sortkey = ymd[0:4] + ymd[5:7] + ymd[8:10] #年月日でのソート用

    tmp = {             #辞書型で格納
        "title": title,
        "summary": summary,
        "link": link,
        "published": ymd,
        "sortkey": sortkey
        }

    news.append(tmp)  #各記事の情報をリストに追加

    #データフレームに変換してcsvファイルで保存
    news_df = pd.DataFrame(news)
    pprint.pprint(news_df.head())  #最初の5行を表示してデータ確認 
    filename = s + ".csv"
    news_df.to_csv(filename, encoding='utf-8-sig', index=False) 